In [94]:
import os
import json
import pygments
from gensim.models import Word2Vec 
import numpy as np
from scipy import spatial

In [42]:
with open('./index/file_key.txt', 'r') as f:
    id_to_f = json.loads(f.read())['id-to-f']
with open('./temp.json', 'r') as f:
    function_defs = json.loads(f.read())

In [53]:

class NameFormatter(pygments.formatter.Formatter):
    def __init__(self):
        self.names = []

    def format(self, tokensource, outfile):
        for _type, value in tokensource:
            if _type[0] == "Name" or _type[0] == "Keyword":
                self.names.append(value)
    
    def get_tok_arr(self):
        return self.names

In [58]:
f_to_toklist = {}
sentences = []
for f in function_defs:
    with open(os.path.join('./raw', f.replace('.json', '.py'))) as fr:
        line_arr = fr.readlines()
        for fd in function_defs[f]:
            for s, e in zip(function_defs[f][fd]['lineno'], function_defs[f][fd]['line-end']):
                func_body = ''.join(line_arr[s:e+2])                   
                formatter = NameFormatter()                               
                pygments.highlight(func_body, pygments.lexers.PythonLexer(), formatter)
                f_to_toklist[f"{f}|{fd}"] = formatter.get_tok_arr()
                sentences.append(formatter.get_tok_arr())

In [62]:
model = Word2Vec(window = 4, min_count = 3, vector_size = 30)
model.build_vocab(corpus_iterable=sentences)
model.train(corpus_iterable=sentences, total_examples=len(sentences), epochs=15)

(4837122, 6435855)

In [92]:
N = len(sentences)
vec_length = model.wv['float'].shape[0]
vec_mat = np.zeros((N, vec_length))
id_to_f = []
f_to_id = {}
for f in function_defs:
    for fd in function_defs[f]:
        fqname = f"{f}|{fd}"
        f_vec = np.mean(np.asarray([model.wv[t] if t in model.wv else np.zeros(vec_length) for t in f_to_toklist[fqname]]), axis=0)
        vec_mat[len(id_to_f)] = f_vec
        f_to_id[fqname] = len(id_to_f)
        id_to_f.append(fqname)
        

/home/chirag/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/chirag/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [95]:
tree = spatial.KDTree(vec_mat)